In [ ]:
import netpixi
from netpixi.integration.gt import *
from regression.integration.gt import *
from graph_tool import centrality
import pandas as pd
import numpy as np

In [2]:
g = gt_load('sampa.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)

## Betweenness

In [3]:
bc, _ = centrality.betweenness(g)
g.add_vp('betweenness', bc)
gt_data(g).loc[:, :].sort_values('betweenness', ascending=False)

,betweenness
id,
PSMJ,0.048244
ESGS,0.038576
ESGN,0.038576
BLES,0.025985
BNDX,0.025985
...,...
SCHJ,0.000000
SCHM,0.000000
SCHV,0.000000


## Centro e Periferia

In [4]:
import cpnet

from graph_tool import spectral

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [5]:
matrix = spectral.adjacency(g)

algorithm = cpnet.Rombach()
algorithm.detect(matrix)
c = algorithm.get_coreness()

g.add_vp('coreness')
for i, coreness in c.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


## Clustering Coefficient

In [6]:
from graph_tool import clustering

lc = clustering.local_clustering(g)
g.add_vp('clustering', lc)

In [7]:
ntw_metrics = gt_data(g)

In [13]:
df = pd.read_csv('etf_funds/ETFs.csv')

In [14]:
metrics = pd.merge(df[['fund_symbol','fund_sharpe_ratio_3years']], ntw_metrics, left_on='fund_symbol', right_on='id', how='inner')

In [15]:
metrics

,fund_symbol,fund_sharpe_ratio_3years,betweenness,coreness,clustering
0,ANEW,NaN,0.000000,0.070346,1.000000
1,BBRE,0.53,0.001634,0.754310,0.717146
2,BEDZ,NaN,0.000000,0.107143,1.000000
3,BFEB,NaN,0.000005,0.207251,0.993978
4,BFIT,0.87,0.000005,0.163961,0.993978
...,...,...,...,...,...
573,TVIXF,-0.26,0.000000,0.050325,1.000000
574,UCIB,0.44,0.001078,0.238636,0.726930
575,UGA,0.34,0.010368,0.247294,0.744111
576,UGAZF,-0.27,0.010368,0.213745,0.744111


In [16]:
metrics.isna().sum()

fund_symbol                   0
fund_sharpe_ratio_3years    184
betweenness                   0
coreness                      0
clustering                    0
dtype: int64